In [1]:
import requests
import mysql.connector
from mysql.connector import Error
import os

In [2]:
#calling data functions from file = 'mod02_data_functions.py', based on notebook md02
from mod02_data_functions import get_mysql_cursor, list_mysql_databases_and_tables, create_covertype_table, delete_covertype_table,\
fetch_all_covertype_records, get_data_from_api, insert_unique_covertype_data


In [3]:
get_mysql_cursor()

MySQL cursor created successfully


(<mysql.connector.cursor_cext.CMySQLCursor at 0x7f46ea5bba50>,
 <mysql.connector.connection_cext.CMySQLConnection at 0x7f46e8615ad0>)

## Remove previous tables

In [4]:
delete_covertype_table('covertype_data')

MySQL cursor created successfully
Table 'covertype_data' deleted if it existed.


# Create Tables - Raw and Clean data

In [5]:
raw_data_table = 'raw_data'
clean_data_table = 'clean_data'

In [6]:
create_covertype_table(raw_data_table)

MySQL cursor created successfully
Table 'raw_data' created or already exists.


In [7]:
create_covertype_table(clean_data_table)

MySQL cursor created successfully
Table 'clean_data' created or already exists.


# Get data from API, save it into raw data table

In [8]:
#Get batch data from API
group_number = 1 #there are 10 groups in total
batch_data = get_data_from_api(group_number)

Number of records retrieved on this batch: 1162


In [9]:
#Save batch_data into raw data table
insert_unique_covertype_data(batch_data, raw_data_table)

MySQL cursor created successfully
Inserted 1162 new rows into 'raw_data'.


# Pre-processing

1. Data Cleaning
Handle missing values: Although the dataset typically has no missing values, it's good practice to check

2. Remove duplicates: Ensure no repeated rows.

3. Feature Engineering
Normalize distances: Scale distance-related features to reduce skew.

Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Horizontal_Distance_To_Fire_Points
Hillshade interaction: Create a feature like average hil

df['Hillshade_Avg'] = df[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].mean(axis=1)
4. Encoding Categorical Variables
Wilderness_Area and Soil_Type are categorical.
Use One-Hot Encoding if using tree-based models.
Use Label Encoding if using linear models or neural networ

5. Feature Scaling
Apply StandardScaler or MinMaxScaler to numerical features:
Elevation, Aspect, Slope, et
**************************************
6. Train-Test Split
Split the dataset before modeling

from sklearn.model_selection import train_test_split
X = df.drop("Cover_Type", axis=1)
y = df["Cover_Type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
:c.ks.
lshade:.